### 들어가기 앞서


#### OOV 문제 
* 기계가 모르는 단어가 등장하면 그 단어를 단어 집합에 없는 단어란 의미에서 `OOV(Out-Of-Vocabulary)` 또는 `UNK(Unknown Token)` 이라 표현한다. 이와 같이 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황을 `OOV 문제` 라고 한다.

#### 서브워드 분리(Subword segmenation)
* `서브워드 분리` 작업은 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들의 조합으로 구성된 경우가 많기 때문에 이를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 *전처리 작업* 
* 해당 작업을 통해 OOV 나 희귀 단어, 신조어 같은 문제를 완화시킬 수 있다
* 이 책에선 해당 작업을 하는 토크나이저를 `서브워드 토크나이저` 라고 명명.

# Byte Pair Encoding, BPE

* 1994년에 제안된 데이터 압축 알고리즘.
* 후에 자연어 처리의 서브워드 분리 알고리즘으로 응용되었다.

## 1. 작동 방법
* 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식으로 작동.
* 예) `aaabdaaabac` 문자열에 BPE 수행.
```python
'aaabdaaabac' # 가장 많이 등장하는 글자쌍(byte pair)는 'aa',Z 로 치환
'ZabdZabac'   # 'ab' -> 'Y' 
'ZYdZYac'     # 'ZY' -> 'X'
'XdXac'       # 글자쌍 없음. 최종 결과
```

## 2. 자연어 처리에서의 BPE
* 논문 :  https://arxiv.org/pdf/1508.07909.pdf
* 글자(character) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 `Bottom-up` 방식의 접근을 사용한다. 
* 훈련 데이터에 있는 단어들을 글자 또는 유니코드 단위로 단어 집합 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합.

### 2-1 기존 접근
* 임의의 훈련 데이터로 얻어낸 임의의 단어와 빈도수.
    ```python
    # dictionary 
    'low : 5, lower : 2, newest : 6, widest : 3'`

    # vocabulary
    'low, lower, newest, widest' 

    # 'lowest' 가 등장하면 OOV 문제 발생.
```

### 2-2 BPE 알고리즘 사용한 경우
* 앞선 임의의 단어를 글자 단위로 분리.
```python
    # dictionary
    'l o w : 5, l o w e r : 2, n e w e s t : 6, w i d e s t : 3'
    
    # vocabulary
    'l, o, w, e, r, n, w, s, t, i, d' 
```
* BPE의 특징은 알고리즘 동작 횟수를 사용자가 정한다는 점. 여기선 10번 수행한다고 가정.
```python
    # 총 10회 반복했을 때 결과
    # dictionary
    'low : 5, low e r : 2, newest : 6, widest : 3'
    
    # vocabulary
    'l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest' 
    
    # 'lowest' 가 등장하면 'l o w e s t'로 분리 후 'low','est' 두 단어로 인코딩.
```


In [10]:
# 원 논문에서 공개한 코드 참고하여 수정한 코드 
# 위와 같은 딕셔너리로 진행.
dictionary = {'l o w </w>' : 5,
              'l o w e r </w>' : 2,
              'n e w e s t </w>' : 6,
              'w i d e s t </w>' : 3
             }

In [4]:
import collections

# 유니그램의 pair들의 빈도수를 카운트
def get_stats(dic) :
    pairs = collections.defaultdict(int)
    for word, freq in dic.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print("current freqeunce of pairs : ",dict(pairs))
    return pairs

In [5]:
import re

# 딕셔너리 합치기
def merge_dict(pair, v_in) :
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in :
        w_out = p.sub(''.join(pair),word)
        v_out[w_out] = v_in[word]
    return v_out

In [11]:
BPE_codes = {}
BPE_codes_reverse = {}

BPE_counts = 10 

for i in range(BPE_counts) :
    print("Iteration ",i+1)
    pairs = get_stats(dictionary)
    best = max(pairs,key=pairs.get)
    dictionary = merge_dict(best,dictionary)
    
    BPE_codes[best] = i
    BPE_codes_reverse[best[0]+best[1]] = best
    
    print("new merge : {}".format(best))
    print("dictionary : {}".format(dictionary))

Iteration  1
current freqeunce of pairs :  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge : ('e', 's')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}
Iteration  2
current freqeunce of pairs :  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge : ('es', 't')
dictionary : {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}
Iteration  3
current freqeunce of pairs :  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'est'): 6, ('est', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 

In [13]:
print(dictionary)

{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


### OOV 대처하기

In [18]:
# split word and make pair
def get_pairs(word) :
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char,char))
        prev_char = char
    return pairs

In [49]:
# encode
def encode(origin) :
       
    word = tuple(origin) + ('</w>',)
    print("word split into characters : ",word)
    
    pairs = get_pairs(word)
    
    if not word :
        return origin
    
    cnt = 0
    while True :
        cnt += 1
        print("Iteration ",cnt)
        print("bigram in word : ",pairs)
        bigram = min(pairs,key = lambda pair:BPE_codes.get(pair,float('INF')))
        print("candidate for merging : ",bigram)
        if bigram not in BPE_codes :
            print("Candidate not in BPE codes, algorithm stop")
            break
        
        first,second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try :
                j = word.index(first,i)
                new_word.extend(word[i:j])
                i = j
            except :
                new_word.extend(word[i:])
                break
                
            if word[i] == first and i < len(word)-1 and word[i+1] == second :
                new_word.append(first+second)
                i += 2
            else :
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging : ",word)
        if len(word) == 1:
            break
        else :
            pairs = get_pairs(word)

    if word[-1] == '</w>' :
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)
    return word

In [50]:
encode("lowest")

word split into characters :  ('l', 'o', 'w', 'e', 's', 't', '</w>')
Iteration  1
bigram in word :  {('w', 'e'), ('e', 's'), ('s', 't'), ('l', 'o'), ('t', '</w>'), ('o', 'w')}
candidate for merging :  ('e', 's')
word after merging :  ('l', 'o', 'w', 'es', 't', '</w>')
Iteration  2
bigram in word :  {('l', 'o'), ('w', 'es'), ('es', 't'), ('t', '</w>'), ('o', 'w')}
candidate for merging :  ('es', 't')
word after merging :  ('l', 'o', 'w', 'est', '</w>')
Iteration  3
bigram in word :  {('l', 'o'), ('est', '</w>'), ('w', 'est'), ('o', 'w')}
candidate for merging :  ('est', '</w>')
word after merging :  ('l', 'o', 'w', 'est</w>')
Iteration  4
bigram in word :  {('l', 'o'), ('w', 'est</w>'), ('o', 'w')}
candidate for merging :  ('l', 'o')
word after merging :  ('lo', 'w', 'est</w>')
Iteration  5
bigram in word :  {('lo', 'w'), ('w', 'est</w>')}
candidate for merging :  ('lo', 'w')
word after merging :  ('low', 'est</w>')
Iteration  6
bigram in word :  {('low', 'est</w>')}
candidate for mergi

('low', 'est')

In [51]:
encode("loki")

word split into characters :  ('l', 'o', 'k', 'i', '</w>')
Iteration  1
bigram in word :  {('l', 'o'), ('i', '</w>'), ('k', 'i'), ('o', 'k')}
candidate for merging :  ('l', 'o')
word after merging :  ('lo', 'k', 'i', '</w>')
Iteration  2
bigram in word :  {('i', '</w>'), ('k', 'i'), ('lo', 'k')}
candidate for merging :  ('i', '</w>')
Candidate not in BPE codes, algorithm stop


('lo', 'k', 'i')

In [52]:
encode("highing")

word split into characters :  ('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')
Iteration  1
bigram in word :  {('h', 'i'), ('i', 'g'), ('g', 'h'), ('n', 'g'), ('i', 'n'), ('g', '</w>')}
candidate for merging :  ('h', 'i')
Candidate not in BPE codes, algorithm stop


('h', 'i', 'g', 'h', 'i', 'n', 'g')

## 추가적인 알고리즘

`그런게 있다 정도로 알고만 넘어감.`

### 1. Wordpiece Model

* 논문 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf 
* 구글이 위 Wordpiece Model을 변형하여 번역기에 사용했다는 논문 : https://arxiv.org/pdf/1609.08144.pdf

* BPE의 변형 알고리즘. WPM.
* BPE 가 빈도수 기반에 가장 많이 등장하는 쌍을 병합하는 것과 달리, 병합되었을 때 코퍼스의 우도(likelihood)를 가장 높이는 쌍을 병합.

* 예시

    `WPM을 수행하기 이전의 문장: Jet makers feud over seat width with big orders at stake`
    
    `WPM을 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake`
    
* 언더바(_) 를 붙여 기존의 띄어쓰기인지 구분자 역할의 띄어쓰기 인지 구분. + 복원 가능.
* BERT를 훈련하기 위해서 사용되기도 했다.


### 2. Unigram Language Model Tokenizer


* 논문 : https://arxiv.org/pdf/1804.10959.pdf
* 서브워드들에 대해서 손실(loss)을 계산
* 손실 : 해당 서브워드가 단어 집합에서 제거되었을 경우, 코퍼스의 우도가 감소하는 정도.
* 최악의 영향을 주는 10~20%의 토큰을 제거하며 원하는 단어 집합의 크기가 될 때 까지 반복.